In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

from typing import Union
from pathlib import Path

import numpy as np
import h5py
import matplotlib.pyplot as plt
from skimage.filters import threshold_local
from skimage.morphology import skeletonize_3d, skeletonize
import porespy

from superresolution import SIM_3D_Data, SpatialUnit, cast_to_float32
from visualizeSIM import plot_overlay, plot_selection_box, plot_slices

In [2]:
def read_SIM(filepath: Union[str, Path]):
    #filepath = Path(filepath)
    file = h5py.File(filepath, 'r')
    data = file['Data']
    scale = tuple(data.attrs['pixel_sizes'])
    channels = data.attrs['channels']

    return [SIM_3D_Data(data[i, ...], scale, SpatialUnit.M, c) for i, c in enumerate(channels)]

In [3]:
SIM_data = read_SIM('./Data/Fed_X63_Z3_SIM.h5')
SIM_data = [cast_to_float32(single_channel_data) for single_channel_data in SIM_data]

In [98]:
x_range, y_range, z_range = (800, 950), (1850, 2000), None
cropped_SIM = [SIM.crop_data(x_range, y_range, z_range) for SIM in SIM_data]


fig, axes = plt.subplots(1, 2, figsize = (15, 8))
cmaps = ('Greens', 'Reds')

for ax, SIM, cmap in zip(axes, SIM_data, cmaps):
    ax = plot_overlay(SIM, ax, cmap = cmap)
    
    ax.set_title(f'{SIM.name.capitalize()} overlay')
    ax = plot_selection_box(x_range, y_range, ax, linewidth = 1, edgecolor = 'r', facecolor = 'none')
fig.suptitle(f'ROI for $x \in {x_range}$; $y \in {y_range}$')
fig.tight_layout()

In [99]:
for ax, SIM, cmap in zip(axes, cropped_SIM, cmaps):
    ax = plot_overlay(SIM, ax, cmap = cmap)
    
    ax.set_title(f'{SIM.name.capitalize()} overlay')
    #ax = plot_selection_box(x_range, y_range, ax, linewidth = 1, edgecolor = 'r', facecolor = 'none')
fig.suptitle(f'ROI for $x \in {x_range}$; $y \in {y_range}$')
fig.tight_layout()

In [101]:
channel = 1
plt.rc('font', size = 12)
fig, ax = plot_slices(cropped_SIM[channel], 5, 5, cmap = cmaps[channel])
fig.suptitle(f'{cropped_SIM[channel].name.capitalize()} channel, raw')
fig.tight_layout()

In [102]:
import scipy.ndimage as ndimage

interp_SIM = [SIM[15:35, :, :].apply(lambda vol: ndimage.zoom(vol, (1.3252e-07/3.130295e-08, 1, 1))) for SIM in cropped_SIM]
for SIM in interp_SIM:
    SIM.scale = (3.130295e-08, 3.130295e-08, 3.130295e-08)

In [103]:
bin_int_SIM = [SIM.apply_along_axis(lambda img: img > threshold_local(img, 23)) for SIM in interp_SIM]

In [104]:
bin_int_SIM[0].shape

(85, 150, 150)

In [106]:
channel = 1
plt.rc('font', size = 12)
fig, ax = plot_slices(bin_int_SIM[channel], 5, 5, cmap = cmaps[channel])
fig.suptitle(f'{bin_int_SIM[channel].name.capitalize()} channel, locally binarized')
fig.tight_layout()

In [109]:
import pyvista as pv
channel = 1
cmap = ['kbc', 'fire'][channel]
pvgrid = bin_int_SIM[channel].to_pvUniformGrid()

plotter = pv.Plotter(notebook = False)
plotter.add_mesh(pvgrid.threshold(value = 0.5).extract_geometry().smooth(n_iter = 500).elevation(), cmap = cmap, lighting = True, opacity = 1.0)
plotter.show_grid()
plotter.show()

In [17]:
thickness_maps = [SIM.apply(lambda vol: porespy.filters.local_thickness(vol, sizes = 40)) for SIM in bin_int_SIM]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [18]:
plt.hist(thickness_maps[0].data.flatten(), bins = 30)


(array([1328355.,       0.,       0.,       0.,    3942.,    3944.,
              0.,    1544.,   12727.,   14038.,    2907.,   24674.,
          18267.,   25492.,   58825.,    7821.,  137879.,   46339.,
          12474.,   15257.,  104586.,   27147.,       0.,   26489.,
          24587.,    8535.,       0.,    1698.,    2860.,    2113.]),
 array([0.        , 0.23894886, 0.47789772, 0.71684658, 0.95579544,
        1.19474431, 1.43369317, 1.67264203, 1.91159089, 2.15053975,
        2.38948861, 2.62843747, 2.86738633, 3.1063352 , 3.34528406,
        3.58423292, 3.82318178, 4.06213064, 4.3010795 , 4.54002836,
        4.77897722, 5.01792609, 5.25687495, 5.49582381, 5.73477267,
        5.97372153, 6.21267039, 6.45161925, 6.69056811, 6.92951697,
        7.16846584]),
 <BarContainer object of 30 artists>)

In [54]:
import pyvista as pv
channel = 1
pvgrid = thickness_maps[channel].to_pvUniformGrid()
plotter = pv.Plotter(notebook = False)
plotter.add_mesh(pvgrid.threshold(value = 0.1).extract_geometry().smooth(n_iter = 200), cmap = 'fire', lighting = True, opacity = 1.0)
plotter.show()

In [50]:
parts = [SIM.apply(lambda vol: porespy.filters.snow_partitioning(vol, r_max = 10).regions) for SIM in bin_int_SIM]

KeyboardInterrupt: 

In [10]:
plotter = pv.Plotter(notebook = False)
channel = 0
pvgrid = parts[channel].to_pvUniformGrid()

plotter.add_mesh(pvgrid, cmap = 'kbc', lighting = True, opacity = 1.0)
plotter.show()

NameError: name 'parts' is not defined

In [110]:
import itk
def itk_skeleton(sim):
    skeleton = itk.BinaryThinningImageFilter3D.New(itk.GetImageFromArray(np.float32(sim.data)))
    return itk.GetArrayFromImage(skeleton)

def itk_thickness(sim):
    skeleton = itk.MedialThicknessImageFilter3D.New(itk.GetImageFromArray(np.float32(sim.data)))
    return itk.GetArrayFromImage(skeleton)


itk_skel = [SIM.apply(itk_skeleton) for SIM in bin_int_SIM]

In [77]:
plotter = pv.Plotter(notebook = False)
channel = 1
pvgrid = itk_skel[channel].to_pvUniformGrid()

plotter.add_mesh(pvgrid.threshold(0.5).elevation(), cmap = 'fire', lighting = True, opacity = 1.0)
plotter.show()

In [114]:
plotter = pv.Plotter(notebook = False)
#pvgrid_actin = itk_skel[0].to_pvUniformGrid()
pvgrid_desmin = itk_skel[1].to_pvUniformGrid()
#plotter.add_mesh(pvgrid_actin.threshold(value = 0.5).elevation().extract_geometry().smooth(n_iter = 500), cmap = 'kbc', lighting = True, opacity = 1.0)
plotter.add_mesh(pvgrid_desmin.threshold(value = 0.5).elevation().extract_geometry().smooth(n_iter = 500), cmap = 'fire', lighting = True, opacity = 1.0)
plotter.show()

In [76]:
itk_thick = [SIM.apply(itk_thickness) for SIM in bin_int_SIM]

In [48]:
plotter = pv.Plotter(notebook = False)
channel = 0
pvgrid = itk_thick[channel].to_pvUniformGrid()

plotter.add_mesh(pvgrid.threshold(1e-3), cmap = 'kbc', lighting = True, opacity = 1.0)
plotter.show()

NameError: name 'itk_thick' is not defined

In [91]:
plt.hist(itk_thick[0].data[itk_thick[0].data>0], bins = 20)

(array([1.835e+03, 7.340e+02, 0.000e+00, 2.150e+02, 1.305e+03, 1.171e+03,
        1.800e+02, 8.830e+02, 1.075e+03, 1.050e+02, 3.860e+02, 5.900e+01,
        2.250e+02, 4.200e+01, 2.900e+01, 1.000e+00, 2.000e+01, 0.000e+00,
        2.000e+00, 3.000e+00]),
 array([ 2.       ,  2.4830952,  2.9661903,  3.4492855,  3.9323807,
         4.415476 ,  4.898571 ,  5.381666 ,  5.8647614,  6.3478565,
         6.8309517,  7.314047 ,  7.797142 ,  8.280237 ,  8.763332 ,
         9.246428 ,  9.729523 , 10.212618 , 10.695713 , 11.178808 ,
        11.661903 ], dtype=float32),
 <BarContainer object of 20 artists>)

In [92]:
plt.hist(itk_thick[1].data[itk_thick[1].data>0], bins = 20)

(array([4685.,    0., 2250.,  530.,    0., 3290., 2126.,  267.,    0.,
         982.,  958.,   66.,    8.,  169.,    0.,   90.,   15.,    9.,
           0.,    9.]),
 array([ 2. ,  2.4,  2.8,  3.2,  3.6,  4. ,  4.4,  4.8,  5.2,  5.6,  6. ,
         6.4,  6.8,  7.2,  7.6,  8. ,  8.4,  8.8,  9.2,  9.6, 10. ],
       dtype=float32),
 <BarContainer object of 20 artists>)